### <b>Detectron2 설치</b>

In [ ]:
!pip install pyyaml==5.1

# PyTorch 1.9.0 버전에 맞는 Detectron2 설치하기
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

# 설치가 완료되면 런타임 재시작하기

In [ ]:
# PyTorch 설치 정보 확인 (1.9.0 버전이 본 실습의 기본 설정)
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.9.0+cu102 True


In [ ]:
# Detectron2 로깅(logging) 설정
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# 일반적으로 많이 사용되는 라이브러리 불러오기
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# 일반적으로 많이 사용되는 Detectron2 라이브러리 불러오기
from detectron2 import model_zoo # Detectron2 모델
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

### <b>사전 학습된 Detectron2 모델 불러오기</b>


* Detectron2 환경설정(config) 정보를 생성한 뒤에 DefaultPredictor를 이용해 이미지에 대하여 추론합니다.
* [Detectron2 기본 환경설정 정보 확인하기](https://detectron2.readthedocs.io/en/latest/modules/config.html#config-references)
    * 참고로 cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST의 기본 값은 0.05입니다.

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# Detectron2 모델을 찾아 학습된 가중치 불러오기
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

model_final_280758.pkl: 167MB [00:09, 17.2MB/s]                           
The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [ ]:
outputs = predictor(im) # 이미지를 모델에 넣어 결과 계산하기

# 결과 출력하기
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# Visualizer를 이용해 이미지와 함께 예측 결과를 출력하기
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

### <b>데이터셋 등록하기</b>

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("my_coco_val2017_1000", {}, "my_coco_val2017_1000/annotations/instances.json", "my_coco_val2017_1000/data")

### <b>평가 진행하기</b>

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("my_coco_val2017_1000") # output_dir 인자 값이 없으면 오류 발생할 수 있음
val_loader = build_detection_test_loader(cfg, "my_coco_val2017_1000")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

#### <b>Test-Time Augmentation (TTA)</b>

* Detectron2에서의 [TTA 구현](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/test_time_augmentation.py)을 참고할 수 있습니다.
* Detectron2의 [modeling 라이브러리](https://detectron2.readthedocs.io/en/latest/modules/modeling.html#detectron2.modeling.GeneralizedRCNNWithTTA)를 참고할 수 있습니다.
* Detectron2의 [TTA 관련 질문](https://github.com/facebookresearch/detectron2/issues?q=TTA)을 참고할 수 있습니다.

In [ ]:
from detectron2.modeling import GeneralizedRCNNWithTTA

tta_model = GeneralizedRCNNWithTTA(cfg, predictor.model)

In [ ]:
print(inference_on_dataset(tta_model, val_loader, evaluator))

#### <b>다양한 모델로 실험해보기</b>

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
predictor_2 = DefaultPredictor(cfg)
print(inference_on_dataset(predictor_2.model, val_loader, evaluator))

In [ ]:
tta_model_2 = GeneralizedRCNNWithTTA(cfg, predictor_2.model)
print(inference_on_dataset(tta_model_2, val_loader, evaluator))

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
predictor_3 = DefaultPredictor(cfg)
print(inference_on_dataset(predictor_3.model, val_loader, evaluator))

In [ ]:
tta_model_3 = GeneralizedRCNNWithTTA(cfg, predictor_3.model)
print(inference_on_dataset(tta_model_3, val_loader, evaluator))